## Importing tweets

### Library and functions. 

The file personalTokens has my tokens from my academic account in Twitter. 

In [2]:
import numpy as np
import tweepy
import json
import pandas as pd
import pytz
from datetime import datetime
import functions as tf
import pickle
import os
from passwords import *

In [48]:
usersDf=pd.read_excel('Twitter_OutputData/UsersIdsPresidents.xlsx')
data=pd.read_excel('../DisData/Raw/datacandidatos.xlsx', sheet_name='datacandidatos')
data['Twitterhandle']=data['Twitterhandle'].str.replace("@","")

In [49]:
# Filtering the data
data = data.loc[data["Momento"] == 1]
data['Twitterhandle']=data['Twitterhandle'].str.replace("@","")
data = data[['Country_code','Twitterhandle','Electionyear']]
data['Electionyear'] = data['Electionyear'].astype(np.int64)
data= data[data['Electionyear'] > 2008]
data = data[data['Twitterhandle'].notna()]
# Start and end time step 1
data['start_time_value']=data['Electionyear'] - 1
data['end_time_value']=data['Electionyear'] + 1
data.loc[data["Electionyear"] < 2012, "start_time_value"] = 2010
data.loc[data["Electionyear"] == 2009, "end_time_value"] = 2010
data['start_time_value']=data['start_time_value'].apply(str)
data['end_time_value']=data['end_time_value'].apply(str)
data['start_time_value']=np.where(data['start_time_value']=='2010','2010-11-06T00:00:00Z',data['start_time_value']+'-01-01T00:00:00Z')
data['end_time_value']=data['end_time_value']+'-12-30T00:00:00Z'
data.reset_index()

,index,Country_code,Twitterhandle,Electionyear,start_time_value,end_time_value
0,42,ARG,CFKArgentina,2011,2010-11-06T00:00:00Z,2012-12-30T00:00:00Z
1,43,ARG,RICALFONSIN,2011,2010-11-06T00:00:00Z,2012-12-30T00:00:00Z
2,44,ARG,eduardoaduhalde,2011,2010-11-06T00:00:00Z,2012-12-30T00:00:00Z
3,45,ARG,HermesBinner,2011,2010-11-06T00:00:00Z,2012-12-30T00:00:00Z
4,48,ARG,altamirajorge,2011,2010-11-06T00:00:00Z,2012-12-30T00:00:00Z
...,...,...,...,...,...,...
269,761,DOR,luisabinader,2020,2019-01-01T00:00:00Z,2021-12-30T00:00:00Z
270,762,DOR,LeonelFernandez,2020,2019-01-01T00:00:00Z,2021-12-30T00:00:00Z
271,763,DOR,MorenoGuillermo,2020,2019-01-01T00:00:00Z,2021-12-30T00:00:00Z
272,764,DOR,juancohens,2020,2019-01-01T00:00:00Z,2021-12-30T00:00:00Z


In [50]:
data2=data.merge(usersDf, on='Twitterhandle', how='left')
data2=data2[data2['userId'] != 'None']
data2 = data2[data2['userId'].notna()]
data2['tw_election']=data2['Twitterhandle']+data2['Electionyear'].astype(str)
data2=data2.reset_index()
data4=data2.copy()

In [24]:
data4=data2.copy()

,index,Country_code,Twitterhandle,Electionyear,start_time_value,end_time_value,Unnamed: 0,userId,description
0,26,PER,KeikoFujimori,2011,2010-11-06T00:00:00Z,2012-12-30T00:00:00Z,62.0,53189543,Mamá de Kyara y Kaori. Casada con Mark. Fundad...
1,32,PER,KeikoFujimori,2016,2015-01-01T00:00:00Z,2017-12-30T00:00:00Z,62.0,53189543,Mamá de Kyara y Kaori. Casada con Mark. Fundad...
2,270,DOR,LeonelFernandez,2020,2019-01-01T00:00:00Z,2021-12-30T00:00:00Z,367.0,1669631264,Dominicano. Abogado. Presidente de la Repúblic...


In [46]:
# extraer tweets
for index, row in data4.iterrows():
    #for userId in conjunto:
    dcandidato={data4.loc[index,'tw_election']:{'userId':data4.loc[index,'userId'],
       'var_username':data4.loc[index,'Twitterhandle'],
       'tweets':[],'created_at':[],'retweet_count':[],'reply_count':[],'like_count':[],'quote_count':[]}}
    dic_tweets_candidato=tf.get_all_tw_user1(userId = data4.loc[index,'userId'], #funcion nueva
                              var_username = data4.loc[index,'Twitterhandle'],
                              start_time_value = data4.loc[index,'start_time_value'],
                             end_time_value = data4.loc[index,'end_time_value'],
                                    tw_election_value = data4.loc[index,'tw_election'],
                                    dic=dcandidato)
    
    
    tw_election_value = data4.loc[index,'tw_election']
    country = data4.loc[index,'Country_code']
    
    #convert to dataframe
    df_tweets_candidato=pd.DataFrame.from_dict(dic_tweets_candidato[tw_election_value])

    #convert dates
    df_tweets_candidato['created_at'] = df_tweets_candidato['created_at'].apply(lambda a: pd.to_datetime(a).date())
    
    
    #create folder if doesn't exist
    if not os.path.exists(country):
        os.makedirs(country)
        
    
    filename=data4.loc[index,'tw_election']+'.xlsx'
    path_name='./'+country+'/'+filename
    df_tweets_candidato.to_excel(path_name)

KeikoFujimori
2012-04-19 21:59:07+00:00
2011-07-07 19:13:43+00:00
2011-05-19 22:24:03+00:00
2011-05-03 22:37:58+00:00
2011-04-16 13:53:31+00:00
2011-01-03 18:19:30+00:00
KeikoFujimori
2016-11-12 15:56:07+00:00
2016-05-18 16:29:03+00:00
2016-04-10 18:19:19+00:00
2016-03-04 21:27:04+00:00
2015-04-18 14:10:27+00:00
LeonelFernandez
2021-10-02 20:45:23+00:00
2021-08-14 23:41:32+00:00
2021-05-15 16:52:09+00:00
2021-02-11 15:18:49+00:00
2020-12-08 12:30:14+00:00
2020-11-06 20:09:58+00:00
2020-07-06 03:16:51+00:00
2020-06-17 14:30:46+00:00
2020-06-02 00:22:36+00:00
2020-05-10 18:43:03+00:00
2020-04-12 18:42:38+00:00
2020-03-04 03:26:37+00:00
2020-02-13 03:12:50+00:00
2020-01-18 02:32:02+00:00
2019-11-27 14:21:44+00:00
2019-10-23 02:43:32+00:00
2019-10-14 12:17:49+00:00
2019-09-28 14:08:27+00:00
2019-09-10 21:28:16+00:00
2019-08-17 21:09:51+00:00
2019-05-04 16:10:34+00:00
2019-02-06 15:22:05+00:00
